<a href="https://colab.research.google.com/github/Jakeag0/StanfordREU-TurtlebotNavigation/blob/main/CNNclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import required modules from keras
from keras.optimizers import Adam
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation

# straight, left, right
NUMBER_OF_CLASSES = 3
# common for LeNET
IMAGE_WIDTH, IMAGE_HEIGHT = 200, 200

# Build Convolutional Neural Network model 
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), padding="same", input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, kernel_size=(5, 5), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dense(NUMBER_OF_CLASSES))
model.add(Activation("softmax"))

# Optimizer
optimizer = Adam()

# Compile Model
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])

# Fitting the Dataset

# data augmentation by slightly altering images
train_data_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_data_generator.flow_from_directory(
    directory="NASA_REU/training_set",
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    # number of images to be yielded per batch
    batch_size=16,
    class_mode="categorical")

test_data_generator = ImageDataGenerator(rescale=1./255)

test_generator = test_data_generator.flow_from_directory(
    directory="NASA_REU/test_set",
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    # number of images to be yielded per batch
    batch_size=16,
    class_mode="categorical")

model.fit_generator(train_generator, steps_per_epoch=100, epochs=5, validation_data=test_generator, validation_steps=100)

# Saving Model in HDF file format
model.save("CNN_classifier_le_net.h5")